# LASSO

This notebook covers various optimization problems related to the LASSO.

In [3]:
import numpy as np
np.random.seed(0)
X = np.loadtxt("X.csv", delimiter=',')
Y = np.loadtxt("Y.csv")

For a given $X, Y$, here is the squared error loss

In [4]:
import regreg.api as rr
loss = rr.squared_error(X, Y)
loss

affine_smooth(quadratic((442,), coef=1, Q=None, Qdiag=False, offset=[ 151.   75.  141.  206.  135.   97.  138.   63.  110.  310.  101.   69.
  179.  185.  118.  171.  166.  144.   97.  168.   68.   49.   68.  245.
  184.  202.  137.   85.  131.  283.  129.   59.  341.   87.   65.  102.
  265.  276.  252.   90.  100.   55.   61.   92.  259.   53.  190.  142.
   75.  142.  155.  225.   59.  104.  182.  128.   52.   37.  170.  170.
   61.  144.   52.  128.   71.  163.  150.   97.  160.  178.   48.  270.
  202.  111.   85.   42.  170.  200.  252.  113.  143.   51.   52.  210.
   65.  141.   55.  134.   42.  111.   98.  164.   48.   96.   90.  162.
  150.  279.   92.   83.  128.  102.  302.  198.   95.   53.  134.  144.
  232.   81.  104.   59.  246.  297.  258.  229.  275.  281.  179.  200.
  200.  173.  180.   84.  121.  161.   99.  109.  115.  268.  274.  158.
  107.   83.  103.  272.   85.  280.  336.  281.  118.  317.  235.   60.
  174.  259.  178.  128.   96.  126.  288.   88.  292.  

The object `loss` is an instance of `regreg.smooth.affine_smooth` the representation of a smooth function in `regreg` composed with a linear transformation. Its 
most important API piece is `smooth_objective` which evaluates the function, its gradient or both.

In [5]:
value, score_at_zero = loss.smooth_objective(np.zeros(loss.shape), 'both')
value

6425460.4999999991

In [6]:
score_at_zero, X.T.dot(X.dot(np.zeros(loss.shape)) - Y)

(array([-304.18307453,  -69.71535568, -949.43526038, -714.7416437 ,
        -343.25445189, -281.78459335,  639.14527932, -696.88303009,
        -916.13872282, -619.22282068]),
 array([-304.18307453,  -69.71535568, -949.43526038, -714.7416437 ,
        -343.25445189, -281.78459335,  639.14527932, -696.88303009,
        -916.13872282, -619.22282068]))

The LASSO uses an $\ell_1$ penalty in "Lagrange" form:
$$
\text{minimize}_{\beta} \frac{1}{2} \|Y-X\beta\|^2_2 + \lambda \|\beta\|_1.
$$

In [7]:
penalty = rr.l1norm(10, lagrange=200.)
print ('penalty:', str(penalty))
penalty

('penalty:', 'l1norm((10,), lagrange=200.000000, offset=None)')


l1norm((10,), lagrange=200.000000, offset=None)

The object penalty is an instance of `regreg.atoms.seminorm`. The main API used in `regreg`
is the method `proximal` which computes the proximal mapping of the object. In `regreg`, an `atom` generally means it has a simple proximal map.

The proximal mapping of the function 
$$
f(\beta) = \lambda \|\beta\|_1
$$
is
$$
\text{prox}_{f, \epsilon}(z) = \text{argmin}_{\beta} \left[\frac{\epsilon}{2}\|\beta-z\|^2_2 + f(\beta)\right].
$$

See [this document](https://web.stanford.edu/~boyd/papers/pdf/prox_algs.pdf) for a brief review of proximal maps.

When $f$ is as above, this is the soft-thresholding map
$$
\text{prox}_{f,\epsilon}(z)_i = 
\begin{cases}
\text{sign}(z_i)(|z_i| - \lambda / \epsilon) & |z_i| > \lambda  / \epsilon \\
0 & \text{otherwise.}
\end{cases}
$$

More generally, we might want to solve
$$
\text{minimize}_{\beta} \left[\frac{C}{2} \|\beta-\mu\|^2_2 + \eta^T\beta + \gamma + f(\beta)\right]
$$
which can easily done if we know the proximal mapping.

In `regreg`, objects $Q$ of the form
$$
Q(\beta) =  \frac{C}{2} \|\beta-\mu\|^2_2 + \eta^T\beta + \gamma
$$
are represented instances of `rr.identity_quadratic`.

In [8]:
Z = np.random.standard_normal(penalty.shape)
penalty.lagrange = 0.1
epsilon = 0.4
quadratic_term = rr.identity_quadratic(epsilon, Z, 0, 0)
penalty.proximal(quadratic_term) - penalty.solve(quadratic_term)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [9]:
threshold = penalty.lagrange / epsilon
soft_thresh_Z = np.sign(Z) * (np.fabs(Z) - threshold) * (np.fabs(Z) > threshold)
soft_thresh_Z

array([ 1.51405235,  0.15015721,  0.72873798,  1.9908932 ,  1.61755799,
       -0.72727788,  0.70008842,  0.        ,  0.        ,  0.1605985 ])

The objects `loss` and `penalty` are combined to form the LASSO objective above. 
This is the canonical problem that we want to solve:
$$
\text{minimize}_{\beta} f(\beta) + g(\beta)
$$
where $f$ is a smooth convex function (i.e. we can compute its value and its gradient)
and $g$ is a function whose proximal map is easy to compute.

The object `rr.simple_problem` requires its first argument to have a `smooth_objective`
method and its second argument to have a `solve` method that solves
$$
\text{minimize}_{\beta} g(\beta) + Q(\beta)
$$
where $Q$ is a quadratic of the above form. If $g$ has a `proximal` method, this step
just calls the proximal mapping.

In [10]:
penalty.lagrange = 200.
problem_lagrange = rr.simple_problem(loss, penalty)
problem_lagrange

In [11]:
coef_lagrange = problem_lagrange.solve(tol=1.e-12)
print(coef_lagrange)

[   0.           -0.          479.01574743  149.17368192   -0.           -0.
  -71.22719502    0.          415.33632118    0.        ]


In [12]:
implied_bound = np.fabs(coef_lagrange).sum()
print(implied_bound)

1114.75294555


In [13]:
bound_constraint = rr.l1norm(10, bound=implied_bound)
bound_constraint

l1norm((10,), bound=1114.752946, offset=None)

In [14]:
problem_bound = rr.simple_problem(loss, bound_constraint)
problem_bound

In [15]:
coef_bound = problem_bound.solve(tol=1.e-12)
print(coef_bound)

[  -0.            0.          479.01516494  149.1742413     0.            0.
  -71.22753915   -0.          415.33600015   -0.        ]


In [16]:
np.linalg.norm(coef_bound - coef_lagrange) / np.linalg.norm(coef_lagrange)

1.4266132415834472e-06

## Comparison to `sklearn`

The objective function is differs from `sklearn.linear_model.Lasso` by a factor of $1/n$.

In [17]:
from sklearn.linear_model import Lasso
clf = Lasso(alpha=penalty.lagrange / X.shape[0])
sklearn_soln = clf.fit(X, Y).coef_
sklearn_soln

array([   0.        ,   -0.        ,  479.04638168,  149.15882396,
         -0.        ,   -0.        ,  -71.20460087,    0.        ,
        415.33678368,    0.        ])

In [18]:
Xtiming = np.random.standard_normal((2000, 4000))
Ytiming = np.random.standard_normal(2000)
lagrange = np.fabs(Xtiming.T.dot(Ytiming)).max() * 0.6

In [19]:
%%timeit
clf = Lasso(alpha=lagrange / Xtiming.shape[0])
sklearn_soln = clf.fit(Xtiming, Ytiming).coef_

1 loops, best of 3: 455 ms per loop


In [20]:
%%timeit
loss = rr.squared_error(Xtiming, Ytiming)
penalty = rr.l1norm(Xtiming.shape[1], lagrange=lagrange)
rr.simple_problem(loss,penalty).solve(tol=1.e-12)

1 loops, best of 3: 663 ms per loop


In [21]:
loss_t = rr.squared_error(Xtiming, Ytiming)
penalty_t = rr.l1norm(Xtiming.shape[1], lagrange=lagrange)
soln1 = rr.simple_problem(loss_t, penalty_t).solve(tol=1.e-6)
clf = Lasso(alpha=lagrange / Xtiming.shape[0])
soln2 = clf.fit(Xtiming, Ytiming).coef_
print (soln1 != 0).sum(), (soln2 != 0).sum()
np.linalg.norm(soln1 - soln2) / np.linalg.norm(soln1)
(loss_t.smooth_objective(soln1, 'func') + np.fabs(soln1).sum() * lagrange, loss_t.smooth_objective(soln2, 'func') + np.fabs(soln2).sum() * lagrange)

72 72


(1031.243283031072, 1031.2492548036926)

In [22]:
sklearn_soln

array([   0.        ,   -0.        ,  479.04638168,  149.15882396,
         -0.        ,   -0.        ,  -71.20460087,    0.        ,
        415.33678368,    0.        ])

In [23]:
np.linalg.norm(sklearn_soln - coef_lagrange) / np.linalg.norm(coef_lagrange)


6.2369887653135822e-05

## Elastic net

The elastic net differs from the LASSO only by addition of a quadratic term.
In `regreg`, both smooth functions and atoms have their own quadratic term that
is added to the objective before solving the problem. 

The `identity_quadratic` is specified as $Q$ above:
$$
Q(\beta) = \frac{C}{2} \|\beta-\mu\|^2_2 + \eta^T\beta + \gamma
$$
with $C$ the first argument, $\mu$ the second, $\eta$ the third and $\gamma$ the fourth.

In [24]:
enet_term = rr.identity_quadratic(0.5,0,0,0)
enet_term

identity_quadratic(0.500000, 0.0, 0.0, 0.000000)

In [25]:
penalty_enet = rr.l1norm(10, lagrange=200., quadratic=enet_term)
penalty_enet

l1norm((10,), lagrange=200.000000, offset=None, quadratic=identity_quadratic(0.500000, 0.0, 0.0, 0.000000))

In [26]:
problem_enet = rr.simple_problem(loss, penalty_enet)
enet_lagrange = problem_enet.solve(min_its=200, tol=1.e-12)
enet_lagrange

array([   0.        ,   -0.        ,  330.99470749,  153.30113271,
          0.        ,    0.        ,  -90.63802712,   40.66226508,
        286.38998267,   37.24626444])

Quadratic terms can also be added to problems as the first argument to `solve`.

In [27]:
problem_lagrange.solve(enet_term, min_its=200, tol=1.e-12)

array([   0.        ,   -0.        ,  330.99470749,  153.30113271,
          0.        ,    0.        ,  -90.63802712,   40.66226508,
        286.38998267,   37.24626444])

Objects like `enet_term` are ubiquitous in `regreg` because it is a package
that uses proximal gradient methods to solve problems. Hence, it is repeatedly solving problems like
$$
\text{minimize}_{\beta} \frac{C}{2} \|z-\beta\|^2_2 + {\cal P}(\beta).
$$

It therefore manipulates these objects in the course of solving the problem.
The arguments to `rr.identity_quadratic` determine functions like
$$
\beta \mapsto \frac{C}{2} \|\beta - \mu\|^2_2 + \beta^T\eta + \gamma.
$$



In [28]:
C = 0.5 
mu = np.arange(4)
eta = np.ones(4)
gamma = 2.3

iq = rr.identity_quadratic(C, mu, eta, gamma)
str(iq)

'identity_quadratic(0.500000, array([0, 1, 2, 3]), array([ 1.,  1.,  1.,  1.]), 2.300000)'

In [29]:
beta = -np.ones(4)
iq.objective(beta, 'func'), 0.5*C*((beta-mu)**2).sum() + (beta*eta).sum() + gamma

(5.7999999999999998, 5.7999999999999998)

The arguments $\mu$ is the `center` and $\eta$ is the `linear_term`, the argument $\gamma$ is `constant` which seems somewhat unnecessary but is sometimes useful to track through computations.
such that `center` is 0.

In [30]:
str(iq.collapsed())

'identity_quadratic(0.500000, 0.0, array([ 1. ,  0.5,  0. , -0.5]), 5.800000)'

As atoms and smooth functions have their own such quadratic terms, one sometimes collects
them to form an overall quadratic term

In [31]:
iq2 = rr.identity_quadratic(0.3, eta, mu, -2.1)
iq2

identity_quadratic(0.300000, array([ 1.,  1.,  1.,  1.]), array([0, 1, 2, 3]), -2.100000)

In [32]:
str(iq+iq2)

'identity_quadratic(0.800000, 0.0, array([ 0.7,  1.2,  1.7,  2.2]), 4.300000)'

In [33]:
iq.collapsed()

identity_quadratic(0.500000, 0.0, array([ 1. ,  0.5,  0. , -0.5]), 5.800000)

## Dual problems

The LASSO or Elastic Net can often be solved by solving an associated dual problem.
There are various ways to construct such problems. 

One such way is to write our elastic net problem as
$$
\text{minimize}_{\beta} f(\beta) + g(\beta)
$$
where
$$
\begin{aligned}
f(\beta) &= \frac{1}{2} \|Y-X\beta\|^2_2 + \frac{C}{2} \|\beta\|^2_2 \\
g(\beta) &= \lambda \|\beta\|_1.
\end{aligned}
$$

Then, we duplicate the variable $\beta$ yielding
$$
\text{minimize}_{\beta_1,\beta_2:\beta_1=\beta_2} f(\beta_1) + g(\beta_2)
$$
and introduce the Lagrangian
$$
L(\beta_1,\beta_2,u) = f(\beta_1) + g(\beta_2) + u^T(\beta_1-\beta_2).
$$

The dual problem is constructed by minimizing over $(\beta_1,\beta_2)$ which yields a function of
$u$:
$$
\inf_{\beta_1,\beta_2}L(\beta_1,\beta_2,u) = -f^*(-u) - g^*(u)
$$
where 
$$
f^*(u) = \sup_{\beta} \beta^Tu - f(\beta)
$$
is the convex conjugate of $f$.

The dual problem, written as a minimization problem is
$$
\text{minimize}_{u} f^*(-u) + g^*(u).
$$

In the elastic net case, 
$$
g^*(u) = I^{\infty}(\|u\|_{\infty} \leq \lambda)
$$
and
$$
\begin{aligned}
f^*(-u) &= -\inf_{\beta}\left[ \frac{1}{2} \|Y-X\beta\|^2_2 + \frac{C}{2}\|\beta\|^2_2 + u^T\beta\right] \\
\end{aligned}
$$

We see the optimal $\beta$ in computing the infimum aboves satisfies the normal equations
$$
(X^TX + C \cdot I)\beta^*(u,Y) = X^TY - u
$$
or
$$
\beta^*(u,Y) = (X^TX+C \cdot I)^{-1}(X^TY-u).
$$

Therefore,
$$
f^*(-u) = \frac{1}{2} (X^TY-u)^T(X^TX+C \cdot I)^{-1}(X^TY-u) - \frac{1}{2}\|Y\|^2_2.
$$

The function $f^*$ can be evaluated exactly as it is quadratic, though it can also be solved numerically if 
our loss was not squared-error. This is what the class `regreg.api.conjugate` does.

In [34]:
dual_loss = rr.conjugate(loss, negate=True, quadratic=enet_term, tol=1.e-12)
Q = np.linalg.inv(X.T.dot(X) + enet_term.coef * np.identity(10))

def dual_loss_explicit(u):
    z = X.T.dot(Y) - u
    return 0.5 * (z * Q.dot(z)).sum() - 0.5 * (Y**2).sum()

U = np.random.standard_normal(10) * 1
print np.linalg.norm((dual_loss.smooth_objective(U, 'grad') + Q.dot(X.T.dot(Y) - U)))  / np.linalg.norm(dual_loss.smooth_objective(U, 'grad'))
print dual_loss.smooth_objective(U, 'func'), dual_loss_explicit(U)

1.02091801882e-05
-5883988.88044 -5883988.88044


The `negate` option tells `regreg` that the function we want is the conjugate of `loss` composed with
a sign change, i.e. a linear transform.

In [35]:
dual_atom = penalty.conjugate
print str(dual_atom)

supnorm((10,), bound=200.000000, offset=None)


In [36]:
dual_problem = rr.simple_problem(dual_loss, dual_atom)
dual_soln = dual_problem.solve(min_its=50)
dual_soln

array([  87.61073573,  -94.96475265,  200.        ,  200.        ,
         46.2648192 ,   20.2595554 , -200.        ,  200.        ,
        200.        ,  200.        ])

The solution to this dual problem is equal to the negative of the gradient of the objective of our 
elastic net at the solution. This is sometimes referred to as a primal-dual relationship, and is
in effect a restatement of the KKT conditions.

In [37]:
- loss.smooth_objective(enet_lagrange, 'grad') - enet_term.objective(enet_lagrange, 'grad')

array([  87.61073546,  -94.96475199,  200.        ,  200.        ,
         46.26481699,   20.2595532 , -200.        ,  200.        ,
        200.        ,  200.        ])

For the `conjugate` object, `regreg` retains a reference to the minimizer, i.e. the gradient of the
conjugate function. In our problem, this is actually the solution to our elastic net problem, though it
does not have exact zeros.

In [38]:
primal_soln = dual_loss.argmin

In [39]:
primal_soln

array([ -4.16535318e-07,   4.82057016e-07,   3.30994708e+02,
         1.53301133e+02,  -2.22893096e-06,  -2.18321317e-06,
        -9.06380260e+01,   4.06622669e+01,   2.86389983e+02,
         3.72462648e+01])

In [40]:
print np.linalg.norm(primal_soln - enet_lagrange) / np.linalg.norm(enet_lagrange)

8.18565315273e-09


We could alternatively have formed the explicit quadratic function for $f^*(-u)$. Having formed the 
quadratic objective explicitly, we will have to also explicitly solve for the primal solution.

In [41]:
dual_quadratic = rr.quadratic(Q.shape[0], Q=Q, offset=X.T.dot(Y))
dual_problem_alt = rr.simple_problem(dual_quadratic, dual_atom)
dual_soln_alt = dual_problem_alt.solve(min_its=100)
dual_soln_alt

array([  87.61073546,  -94.96475199,  200.        ,  200.        ,
         46.26481699,   20.2595532 , -200.        ,  200.        ,
        200.        ,  200.        ])

In [42]:
primal_soln_alt = -dual_quadratic.smooth_objective(dual_soln_alt, 'grad')
print np.linalg.norm(primal_soln_alt - enet_lagrange) / np.linalg.norm(enet_lagrange)

1.81472493994e-13


## Basis pursuit

Yet another species in the zoology of LASSO problems is the basis pursuit problem
$$
\text{minimize}_{\beta: \|y-X\beta\|_2 \leq \delta} \|\beta\|_1.
$$
This can be written as the sum of two atoms.

In [43]:
l1_part = rr.l1norm(X.shape[1], lagrange=1.)
l1_part

l1norm((10,), lagrange=1.000000, offset=None)

In [44]:
X -= X.mean(0)[None,:]; Y -= Y.mean()
full_soln = np.linalg.pinv(X).dot(Y)
min_norm = np.linalg.norm(Y - X.dot(full_soln))
l2_part = rr.l2norm.affine(X, -Y, bound=1.1*min_norm) # we can't take a bound any smaller than sqrt(RSS)
l2_part

affine_atom(l2norm((442,), bound=1236.697060, offset=array([ -1.13348416e+00,  -7.71334842e+01,  -1.11334842e+01,
         5.38665158e+01,  -1.71334842e+01,  -5.51334842e+01,
        -1.41334842e+01,  -8.91334842e+01,  -4.21334842e+01,
         1.57866516e+02,  -5.11334842e+01,  -8.31334842e+01,
         2.68665158e+01,   3.28665158e+01,  -3.41334842e+01,
         1.88665158e+01,   1.38665158e+01,  -8.13348416e+00,
        -5.51334842e+01,   1.58665158e+01,  -8.41334842e+01,
        -1.03133484e+02,  -8.41334842e+01,   9.28665158e+01,
         3.18665158e+01,   4.98665158e+01,  -1.51334842e+01,
        -6.71334842e+01,  -2.11334842e+01,   1.30866516e+02,
        -2.31334842e+01,  -9.31334842e+01,   1.88866516e+02,
        -6.51334842e+01,  -8.71334842e+01,  -5.01334842e+01,
         1.12866516e+02,   1.23866516e+02,   9.98665158e+01,
        -6.21334842e+01,  -5.21334842e+01,  -9.71334842e+01,
        -9.11334842e+01,  -6.01334842e+01,   1.06866516e+02,
        -9.91334842e+01,   3.786

In [45]:
min_norm*1.1, np.linalg.norm(Y)

(1236.6970603462826, 1618.9530951928127)

The problem can be turned into a problem solvable by `regreg` if we smooth out `l2_part`. This is 
related to the approaches taken by `NESTA` and `TFOCS`.

There are quite a few variations, but one approach is to smooth the `l2_part` and solve a problem with a smoothed conjugate and an $\ell_1$ penalty.


### Smoothing out atoms

In [46]:
small_q1 = rr.identity_quadratic(1.e-4, 0, 0, 0)
l2_part_smoothed = l2_part.smoothed(small_q1)
smoothed_problem = rr.simple_problem(l2_part_smoothed, l1_part)
smoothed_problem

In [47]:
smoothed_soln = smoothed_problem.solve(min_its=10000)
smoothed_soln

array([   0.        ,   -0.        ,  433.87926454,   78.4822789 ,
          0.        ,    0.        ,   -0.        ,    0.        ,
        372.99296915,    0.        ])

### TFOCS

The TFOCS approach similarly smooths atoms, but solves this by adding a small quadratic 
to the objective before solving a dual problem. Formally, `TFOCS` solves a sequence of such
smoothed problems where the quadratic term is updated along the sequence. The center of the quadratic is also updated
along the sequence. 

In [48]:
small_q2 = rr.identity_quadratic(1.e-6, 0, 0, 0)
l1_part2 = rr.l1norm(X.shape[1], lagrange=1., quadratic=small_q2)
linf_smoothed = l1_part2.conjugate
linf_smoothed

smooth_conjugate(l1norm((10,), lagrange=1.000000, offset=None, quadratic=identity_quadratic(0.000001, 0.0, 0.0, 0.000000)),identity_quadratic(0.000001, 0.0, 0.0, 0.000000))

In [49]:
from regreg.affine import scalar_multiply, adjoint
transform, dual_atom = l2_part.dual
full_transform = adjoint(scalar_multiply(transform, -1))
tfocs_problem = rr.simple_problem(rr.affine_smooth(linf_smoothed, full_transform), dual_atom)
tfocs_problem

In [50]:
tfocs_soln = tfocs_problem.solve(tol=1.e-12)

The primal solution is stored in the object `linf_smoothed` as `grad` which was the minimizer
for the conjugate function before applying `full_transform`

In [51]:
primal_soln = linf_smoothed.grad
primal_soln

array([   0.        ,   -0.        ,  433.52869006,   78.08975989,
          0.        ,    0.        ,   -0.        ,    0.        ,
        373.71691565,    0.        ])